In [ ]:
qb = QuantBook()
spy = qb.AddEquity("SPY")

# Define the date ranges
date_ranges = [
    (datetime(2000, 1, 1), datetime(2022, 4, 30)),
    (datetime(2022, 11, 1), datetime(2023, 7, 1)),
    (datetime(2024, 4, 1), datetime(2024, 7, 9)),
    (datetime(2024, 8, 21), datetime(2024, 12, 9))
]

# Retrieve and concatenate histories
histories = []
for start, end in date_ranges:
    history = qb.History(
        spy.Symbol,
        start=start,
        end=end,
        resolution=Resolution.Hour,
        extendedMarketHours=False
    )
    # Filter data to regular market hours
    history = history.loc[spy.Symbol].between_time('09:30', '16:00')
    histories.append(history)

# Concatenate all histories
combined_history = pd.concat(histories)

import pandas as pd
from scipy.stats import t

# Initialize an empty dictionary to store results
fitted_params_by_hour = {}

# Loop through each hour from 10 to 16
for hour in range(10, 17):
    # Format the time string for filtering
    time_str = f'{hour:02}:00'
    
    # Filter data for the given hour
    data_hour_open = history.between_time(time_str, time_str)['open']
    data_hour_close = history.between_time('16:00', '16:00')['close']
    
    # Use only the date part of the index
    data_hour_open.index = data_hour_open.index.date
    data_hour_close.index = data_hour_close.index.date
    
    # Combine the open and close data
    combined_data = pd.concat([data_hour_open, data_hour_close], axis=1)
    combined_data = combined_data.dropna()
    
    # Calculate percent change
    combined_data['percent_change'] = ((combined_data['close'] - combined_data['open']) / combined_data['open'])
    
    # Extract percent change values
    percent_changes = combined_data['percent_change']
    
    # Fit a Student's t-distribution to the percent changes
    fitted_params = t.fit(percent_changes)
    
    # Store the fitted parameters in the dictionary
    fitted_params_by_hour[hour] = fitted_params

# Print the fitted parameters for each hour in the desired format
for hour, params in fitted_params_by_hour.items():
    print(f"{hour-1}: ({params[0]}, {params[1]}, {params[2]}),")